In [42]:
from google.colab import files
uploaded = files.upload()


Saving wearable_sports_health_dataset.csv to wearable_sports_health_dataset.csv


In [43]:
import pandas as pd
import numpy as np


In [44]:
df = pd.read_csv(list(uploaded.keys())[0])
df.columns = df.columns.str.lower().str.strip()

print("Original Columns:", df.columns.tolist())
df.head()


Original Columns: ['record_id', 'athlete_id', 'timestamp', 'heart_rate', 'body_temperature', 'blood_pressure', 'blood_oxygen', 'step_count', 'activity_status', 'latitude', 'longitude', 'secure_transmission_status']


,record_id,athlete_id,timestamp,heart_rate,body_temperature,blood_pressure,blood_oxygen,step_count,activity_status,latitude,longitude,secure_transmission_status
0,1,ATH002,2025-04-10 09:00:00,63,37.1,133/78,96,528,Walking,12.927365,80.116767,0
1,2,ATH009,2025-04-10 09:05:00,71,37.0,128/83,95,330,Running,12.922186,80.115054,0
2,3,ATH001,2025-04-10 09:10:00,131,37.2,116/90,100,858,Resting,12.922204,80.115893,0
3,4,ATH001,2025-04-10 09:15:00,157,37.5,135/75,100,732,Cycling,12.922779,80.112153,0
4,5,ATH006,2025-04-10 09:20:00,73,36.9,112/82,95,667,Cycling,12.926037,80.118071,0


In [45]:
df.rename(columns={
    'date_time': 'timestamp',
    'datetime': 'timestamp',
    'heartrate': 'heart_rate',
    'step_count': 'steps',
    'total_steps': 'steps'
}, inplace=True)


In [46]:
df = df[['timestamp', 'heart_rate', 'steps']]
df.head()


,timestamp,heart_rate,steps
0,2025-04-10 09:00:00,63,528
1,2025-04-10 09:05:00,71,330
2,2025-04-10 09:10:00,131,858
3,2025-04-10 09:15:00,157,732
4,2025-04-10 09:20:00,73,667


In [47]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'] = df['timestamp'].dt.tz_localize('UTC', nonexistent='shift_forward', ambiguous='NaT')


In [48]:
df[['heart_rate', 'steps']] = df[['heart_rate', 'steps']].interpolate(method='linear')
df.fillna(method='ffill', inplace=True)


/tmp/ipython-input-1984737270.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [49]:
df['sleep'] = np.where(
    (df['steps'] == 0) & (df['heart_rate'] < 60),
    1,
    0
)


In [50]:
df.set_index('timestamp', inplace=True)

minute_df = pd.DataFrame({
    'heart_rate': df['heart_rate'].resample('1T').mean(),
    'steps': df['steps'].resample('1T').mean(),
    'sleep': df['sleep'].resample('1T').max()
}).reset_index()


/tmp/ipython-input-1610516864.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'heart_rate': df['heart_rate'].resample('1T').mean(),
/tmp/ipython-input-1610516864.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'steps': df['steps'].resample('1T').mean(),
/tmp/ipython-input-1610516864.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'sleep': df['sleep'].resample('1T').max()


In [51]:
minute_df = minute_df[['timestamp', 'heart_rate', 'steps', 'sleep']]
minute_df.head()


,timestamp,heart_rate,steps,sleep
0,2025-04-10 09:00:00+00:00,63.0,528.0,0.0
1,2025-04-10 09:01:00+00:00,NaN,NaN,NaN
2,2025-04-10 09:02:00+00:00,NaN,NaN,NaN
3,2025-04-10 09:03:00+00:00,NaN,NaN,NaN
4,2025-04-10 09:04:00+00:00,NaN,NaN,NaN


In [52]:
minute_df.to_csv("clean_fitness_data_minute.csv", index=False)
files.download("clean_fitness_data_minute.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
df.head()


,heart_rate,steps,sleep
timestamp,,,
2025-04-10 09:00:00+00:00,63,528,0
2025-04-10 09:05:00+00:00,71,330,0
2025-04-10 09:10:00+00:00,131,858,0
2025-04-10 09:15:00+00:00,157,732,0
2025-04-10 09:20:00+00:00,73,667,0
